# Running PyAeroval

PyAeroval is the most used feature of AeroTools. The point of PyAeroval is that the user, through a "simple" configuration file, can evaluate any combination of model and observation data, and display the result on the [Aeroval web page](https://aeroval.met.no/).

The configuration (config) file can either be a json file that is run with the command `pya aeroval <file path>`, or a python file that calls on PyAeroval with a configuration dictionary. In this tutorial we will use the latter method, as I think it is easier to work with a python files.

We will start by looking at the structure of the configuration dictionary, and then we will look at how to run PyAeroval with said dictionary.

## Configuration Dictionary

The config dictionary is, as you can guess, a Python dictionary. It consists of three "parts"

1. Global options: A set of options for the evaluation, e.g. name, PI, output directory, etc
2. Model config: A dictionary containing information about the models to be used
3. Observation config: A dictionary containing information about the observations to be used

All of this is contained in one dictionary on the form
```
CONFIG = {
    <global option key 1> = <global option value 1>,
    <global option key 2> = <global option value 2>,
    .
    .
    .
    <global option key n> = <global option value n>,




    model_cfg = <dict with model configs>,
    obs_cfg = <dict with observation config>,
}
```

We will see how this is done below

### Global Options

Below you can see a typical set of options used in most of the runs I do with PyAeroval. We will start by first defining some paths and names that should be unique for your configuration

In [ ]:
output_dir = <output_dir>"/data"
coldata_dir = <output_dir>"/coldata"
exp_pi = <your name>
proj_id = "workshop"
exp_id = "workshop"

In [ ]:
CFG = dict(
    # Output directories
    json_basedir=output_dir,
    coldata_basedir=coldata_dir,

    # Run options
    reanalyse_existing=True,        # if True, existing colocated data files will be deleted
    raise_exceptions=True,          # if True, the analysis will stop whenever an error occurs 
    clear_existing_json=False,      # if True, deletes previous output before running

    # Map Options
    add_model_maps=False,           # Adds a plot of the whole map. 
    only_model_maps=False,          # Adds only plot above, without any other evaluation
    filter_name="ALL-wMOUNTAINS",   # Regional filter for analysis
    map_zoom="Europe",              # Zoom level. For EMEP, Europe is typically used
    regions_how="country",          # Calculates statistics for different regions. Typically "country" is used, but that does not work for satellite data

    # Time and Frequency Options
    ts_type="monthly",              # Colocation frequency (no statistics in higher resolution can be computed)
    freqs=["monthly", "yearly"],    # Frequencies that are evaluated
    main_freq="monthly",            # Frequency that is displayed when opening webpage
    periods=["2010"],               # List of years or periods of years that are evaluated. E.g. "2005" or "2001-2020"
    

    # Statistical Options
    obs_remove_outliers=False,
    model_remove_outliers=False,
    colocate_time=False,            # If True, the time of observations and model data will be aligned. Be careful with this option!
    zeros_to_nan=False,
    weighted_stats=True,
    annual_stats_constrained=True,  # If True, stations that does not have data for the whole year are excluded from evaluation
    harmonise_units=True,
    resample_how={"vmro3max": {"daily": {"hourly": "max"}}}, # How to handle Ozone. Used all the time in EMEP

    # Experiment Metadata
    exp_pi=exp_pi,
    proj_id=proj_id,
    exp_id=exp_id,
    exp_name="Evaluation of model data",
    exp_descr=("Evaluation of model data"),
    public=True,
)

We will add one more options before looking at the models and observations: `min_num_obs`. This tells us how many data points are necessary when averaging to a coarser frequency. If that constraint is not met, then the coarser data point will be set to `NaN`.

In [4]:
DEFAULT_RESAMPLE_CONSTRAINTS = dict(
    yearly=dict(monthly=9),
    monthly=dict(
        daily=1,
        weekly=1,
    ),
    daily=dict(hourly=1),
)

CFG["min_num_obs"] = DEFAULT_RESAMPLE_CONSTRAINTS

### Model Configuration

We will use two models here. Both will be defined by a dictionary. They will be so called Aerocom3 data, which is default in PyAerocom, we don't need many arguments to define the models. The difference between the two models we are using is that one is a dataset predefined in PyAerocom, while the other is our own NorESM data.

We start with the predefined dataset. PyAerocom has a set of predefined datasets. There is now easy way of displaying all the datasets as of now, but a catalog is in the making. We are going to use `AEROCOM-MEAN-2x3_AP3-CTRL`. This will be the `model_id` in the below dict

In [ ]:
AEROCOM_MEAN = {
        "model_id": "AEROCOM-MEAN-2x3_AP3-CTRL", 
    }

For the other observation we will use our NorESM data. Here we can make the `model_id` something like *noresm*. As long as this is not a known dataset, PyAeroval will assume that you will provide the path to the data. This is done by giving it another argument `model_data_dir`

In [ ]:
NORESM = {
        "model_id": "noresm",
        "model_data_dir": "/lustre/storeB/project/fou/kl/emep/People/danielh/projects/pyaerocom/workshop/noresm/test_data/noresm",
    }

We can now add these two dictionaries to a common *model dictionary*, and then add this to out configuration dictionary


In [ ]:
MODELS = {
    "AEROCOM-MEAN": AEROCOM_MEAN,
    "NORESM" : NORESM
}

CFG["model_cfg"] = MODELS

### Observations Configuration

Observations can either be observation networks already defined by PyAerocom, or custom network. The problem with observations is that we don't have any conventions like Aerocom, so providing custom observations is not as easy as with models. 

AERONET, EBAS and EEA are the three most used observation networks used in AeroTool. This means that they are "known" by Pyaerocom. By referring to the IDs of the networks, Pyaerocom know exactly how to read the data. We will here only use AERONET:

In [ ]:



AERONET =dict(
        obs_id="AeronetSunV3Lev2.daily",    # Observation ID of know network
        web_interface_name="Aeronet-m",     # Name that is displayed on the webpage
        obs_vars=["od550aer"],              # List of variables that is to be evaluated
        obs_vert_type="Column",             # Observation level
        ts_type="monthly",                  # Frequency of read observations. Evaluation can not be finer than this, for this network
        obs_filters={"latitude": [30, 82],  # Observation filters. Each network can have own filters. Here I've uses lat, lon and alt as examples
                    "longitude": [-30, 90],
                    "altitude": [-200, 5000],
                    }
    )

As we can see, this is a bit more complicated than the model configs. With my comments they should be more or less understandable. Note that again `obs_id` has to be a known network, and as with the models, a catalog of networks is in the works.

In the global options we defined a `min_num_obs`. We can do that here as well. If PyAeroval finds this option in the observation config, as well as in the global options, the one found in the observation config will be prioritized. There are a handful of such options, e.g. outlier limits.



We can now add all of this to out main configuration

In [ ]:
OBS_CFG = {
    "Aeronet-m": AERONET,
}

CFG["obs_cfg"] = OBS_CFG

If you want to see which observation networks are contained in PyAerocom for which aerosol species, you can check out out [observation explorer](https://aeroval.met.no/pages/evaluation/?project=obs&experiment=aerosols-and-gases).

## Running PyAeroval

We have now finally come to the point were we can run this config. While this can be done in this notebook, I instead recommend using the accompanying Python file. But we will here look at the last part of code needed to run the configuration dict

In [ ]:
from pyaerocom.aeroval import EvalSetup, ExperimentProcessor
from pyaerocom import const

print(const.CACHEDIR)               # Prints where to find the caching folder. Not needed but this folder should be emptied now and then, so I like to see where it is


stp = EvalSetup(**CFG)              # Makes a setup object from the dict, that PyAeroval can use
ana = ExperimentProcessor(stp)      # Makes an experiment object
res = ana.run()                     # Runs the experiment


# Example on how to run an experiment with only certain models, obs, and variables. Often used to run only parts of the experiment that has changed
# res = ana.run(model_name=["EMEP"], obs_name=["EEA-m"], var_list=["concCec"])



### Running On PPI

Since PyAeroval evaluation can be quite resource heavy, we recommend either submitting it as a job, or running the python script on a compute node. This can be done by

In [ ]:
%%bash
# on PPI
qlogin -l h_rss=8G,mem_free=8G,h_data=8G # Logs into a compute node with 8GB memory

module load /modules/MET/rhel8/user-modules/fou-kl/aerotools/aerotools

cd <path to the python script>
pya_python pyaeroval_config.py

## Afterword

There are other options, both for the model- and observation setup, as well as the global setup. I've tried to make this config file as slim as possible, while still trying to make it useful. It might seem complex and daunting, but once you have a working config file, you will for the must part just copy  that file, only changing the model and observation setups.